In [9]:
import pandas as pd
import re

In [10]:
filename = input('Enter a class file to grade (i.e. class1 or class1.txt):')
if not '.txt' in filename:
    filename += '.txt'

In [12]:
df = pd.read_csv(filename, names=list(range(30)), header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,N00000021,B,A,C,D,C,C,D,D,C,...,D,A,A,B,D,NaN,NaN,NaN,NaN,NaN
1,N00000022,B,A,D,A,C,B,D,D,C,...,C,A,A,B,A,D,NaN,NaN,NaN,NaN
2,N00000023,NaN,A,D,D,C,B,D,A,C,...,C,A,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,N00000024,C,C,D,D,C,B,NaN,A,C,...,C,A,B,B,NaN,D,NaN,NaN,NaN,NaN
4,N0000002,B,A,D,D,C,B,D,A,C,...,C,A,A,B,D,D,NaN,NaN,NaN,NaN
